In [1]:
#importing libraries
import pandas as pd
import os
import subscript.config as cn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np

In [2]:
# Read in data and set p value Elimination
df = pd.read_csv(os.path.join(cn.clean_dir,'final_aggregated_categories.csv'), dtype = 'unicode')
player_cols =  ['Unnamed: 0','Unnamed: 0.1','player','realm','gear_score','last_login',
            'time_since_login','status']
df = df.drop(player_cols, axis = 1)
dfp = pd.DataFrame()
df = df[df.engagement.astype(float) != 1]
df.head()

,engagement,Alterac Valley,Arathi Basin,Archaeology,Arena,Argent Tournament,Ashran,Battle Dungeon,Battle Raid,Battle for Azeroth,...,Tol Barad,Twin Peaks,Visions of N'Zoth,War Effort,Warsong Gulch,Winter Veil,Wintergrasp,World,World Events,Wrath of the Lich King
0,0.0,3,3,79,6,15,2,0,0,19,...,6,1,0,1,5,5,13,4,1,12
1,0.0,3,2,0,4,0,2,10,3,22,...,0,0,0,3,4,3,5,5,0,11
2,0.0,3,2,0,2,5,2,26,0,23,...,1,0,0,5,2,3,0,5,0,11
3,0.0,3,3,0,2,0,2,1,0,23,...,0,0,0,3,2,3,4,5,0,11
6,0.0,3,2,0,2,12,6,2,0,22,...,0,0,0,3,2,3,8,5,0,11


In [3]:
split = StratifiedShuffleSplit(n_splits = 10, test_size = 0.25, random_state = 17)
for train_index, test_index in split.split(df, df.engagement):
    strat_train = df.iloc[train_index][:]
    strat_test = df.iloc[test_index][:]

y_train = strat_train.engagement
X_train = strat_train.drop('engagement', axis = 1)
y_test = strat_test.engagement
X_test = strat_test.drop('engagement', axis = 1)


In [5]:
feature_selector = SequentialFeatureSelector(RandomForestClassifier(n_jobs=1, n_estimators = 200),
           k_features=50,
           forward=True,
           verbose=2,
           scoring='roc_auc',
           cv=4)
features = feature_selector.fit(np.array(X_train.fillna(0)), y_train)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  86 out of  86 | elapsed:  8.4min finished

[2020-06-23 00:25:30] Features: 1/50 -- score: 0.8275319279890125[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  85 out of  85 | elapsed: 10.6min finished

[2020-06-23 00:36:06] Features: 2/50 -- score: 0.8655168876866334[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  84 out of  84 | elapsed: 15.2min finished

[2020-06-23 00:51:20] Features: 3/50 -- score: 0.8654971178594848[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  

[Parallel(n_jobs=1)]: Done  59 out of  59 | elapsed: 47.3min finished

[2020-06-23 14:07:39] Features: 28/50 -- score: 0.9703734060982768[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   43.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  58 out of  58 | elapsed: 40.1min finished

[2020-06-23 14:47:46] Features: 29/50 -- score: 0.9695868968657949[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   39.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  57 out of  57 | elapsed: 45.1min finished

[2020-06-23 15:32:55] Features: 30/50 -- score: 0.9699325270549611[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   48.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  56 out of  56 | elapsed: 47.5min finished

[2020-06-23 16:20:27] Features: 31/

In [6]:
filtered_features = X_train.columns[list(features.k_feature_idx_)]
print(filtered_features)

Index(['Alterac Valley', 'Battle Dungeon', 'Battle for Gilneas',
       'Battlegrounds', 'Cataclysm Dungeon', 'Cataclysm Raid', 'Character',
       'Collections', 'Cooking', 'Currencies', 'Deepwind Gorge',
       'Draenor Dungeon', 'Dungeons & Raids', 'Eastern Kingdoms', 'Events',
       'Expansion Features', 'Eye of the Storm', 'Feats of Strength',
       'Fishing', 'General', 'Guild Feats of Strength', 'Heart of Azeroth',
       'Honor', 'Island Expeditions', 'Isle of Conquest', 'Kalimdor', 'Legion',
       'Legion Class Hall', 'Legion Dungeon', 'Lich King Dungeon',
       'Lich King Raid', 'Lunar Festival', 'Northrend', 'Outland', 'Pandaria',
       'Pandaria Dungeon', 'Pandaria Raid', 'Pet Battles', 'Pilgrim's Bounty',
       'Professions', 'Promotions', 'Quests', 'Reputation', 'Seething Shore',
       'Temple of Kotmogu', 'The Burning Crusade', 'Visions of N'Zoth',
       'Wintergrasp', 'World', 'Wrath of the Lich King'],
      dtype='object')


In [7]:
clf = RandomForestClassifier(n_estimators=100, random_state=17, max_depth=3)
clf.fit(X_train[filtered_features], y_train)


RandomForestClassifier(max_depth=3, random_state=17)

In [8]:
train_pred = clf.predict_proba(X_train[filtered_features])

print('Accuracy on training set: {}'.format(roc_auc_score(y_train, train_pred[:,1])))

test_pred = clf.predict_proba(X_test[filtered_features].fillna(0))
print('Accuracy on test set: {}'.format(roc_auc_score(y_test, test_pred [:,1])))

Accuracy on training set: 0.9055132718487524
Accuracy on test set: 0.9086960308898895
